5週目2日

### autogen agentchat-より深く行く..

1。マルチモーダル会話
2。構造化された出力
3。Langchainツールの使用
4。チーム

...そして特別なサプライズエクストラピース

In [ ]:
from io import BytesIO
import requests
from autogen_agentchat.messages import TextMessage, MultiModalMessage
from autogen_core import Image as AGImage
from PIL import Image
from dotenv import load_dotenv
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken
from IPython.display import display, Markdown
from pydantic import BaseModel, Field
from typing import Literal

load_dotenv(override=True)


### マルチモーダルの会話

In [ ]:
url = "https://edwarddonner.com/wp-content/uploads/2024/10/from-software-engineer-to-AI-DS.jpeg"

pil_image = Image.open(BytesIO(requests.get(url).content))
img = AGImage(pil_image)
img

In [3]:
multi_modal_message = MultiModalMessage(content=["Describe the content of this image in detail", img], source="User")

In [ ]:
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")

describer = AssistantAgent(
    name="description_agent",
    model_client=model_client,
    system_message="You are good at describing images",
)

response = await describer.on_messages([multi_modal_message], cancellation_token=CancellationToken())
reply = response.chat_message.content
display(Markdown(reply))

### 構造化された出力！

Autogen AgentChatはそれを簡単にします。

In [5]:

class ImageDescription(BaseModel):
    scene: str = Field(description="Briefly, the overall scene of the image")
    message: str = Field(description="The point that the image is trying to convey")
    style: str = Field(description="The artistic style of the image")
    orientation: Literal["portrait", "landscape", "square"] = Field(description="The orientation of the image")


In [ ]:
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")

describer = AssistantAgent(
    name="description_agent",
    model_client=model_client,
    system_message="You are good at describing images in detail",
    output_content_type=ImageDescription,
)

response = await describer.on_messages([multi_modal_message], cancellation_token=CancellationToken())
reply = response.chat_message.content
reply

In [ ]:
import textwrap
print(f"Scene:\n{textwrap.fill(reply.scene)}\n\n")
print(f"Message:\n{textwrap.fill(reply.message)}\n\n")
print(f"Style:\n{textwrap.fill(reply.style)}\n\n")
print(f"Orientation:\n{textwrap.fill(reply.orientation)}\n\n")

### AutogenのLangchainツールを使用します

In [ ]:
# Autogenのラッパー：

from autogen_ext.tools.langchain import LangChainToolAdapter

# Langchainツール：

from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_community.agent_toolkits import FileManagementToolkit
from langchain.agents import Tool


prompt = """Your task is to find a one-way non-stop flight from JFK to LHR in June 2025.
First search online for promising deals.
Next, write all the deals to a file called flights.md with full details.
Finally, select the one you think is best and reply with a short summary.
Reply with the selected flight only, and only after you have written the details to the file."""


serper = GoogleSerperAPIWrapper()
langchain_serper =Tool(name="internet_search", func=serper.run, description="useful for when you need to search the internet")
autogen_serper = LangChainToolAdapter(langchain_serper)
autogen_tools = [autogen_serper]

langchain_file_management_tools = FileManagementToolkit(root_dir="sandbox").get_tools()
for tool in langchain_file_management_tools:
    autogen_tools.append(LangChainToolAdapter(tool))

for tool in autogen_tools:
    print(tool.name, tool.description)

model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
agent = AssistantAgent(name="searcher", model_client=model_client, tools=autogen_tools, reflect_on_tool_use=True)
message = TextMessage(content=prompt, source="user")
result = await agent.on_messages([message], cancellation_token=CancellationToken())
for message in result.inner_messages:
    print(message.content)
display(Markdown(result.chat_message.content))

In [ ]:
# 次に、ファイルを書くためにもう一度エージェントに電話する必要があります

message = TextMessage(content="OK proceed", source="user")

result = await agent.on_messages([message], cancellation_token=CancellationToken())
for message in result.inner_messages:
    print(message.content)
display(Markdown(result.chat_message.content))

### チームの相互作用

In [2]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import  TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool

serper = GoogleSerperAPIWrapper()
langchain_serper =Tool(name="internet_search", func=serper.run, description="useful for when you need to search the internet")
autogen_serper = LangChainToolAdapter(langchain_serper)

model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")


prompt = """Find a one-way non-stop flight from JFK to LHR in June 2025."""


primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    tools=[autogen_serper],
    system_message="You are a helpful AI research assistant who looks for promising deals on flights. Incorporate any feedback you receive.",
)

evaluation_agent = AssistantAgent(
    "evaluator",
    model_client=model_client,
    system_message="Provide constructive feedback. Respond with 'APPROVE' when your feedback is addressed.",
)

text_termination = TextMentionTermination("APPROVE")

# MAX_TURNSを追加してくれたPeter Aに感謝します - それ以外の場合、これはループに入ることができます。

team = RoundRobinGroupChat([primary_agent, evaluation_agent], termination_condition=text_termination, max_turns=20)


In [ ]:
result = await team.run(task=prompt)
for message in result.messages:
    print(f"{message.source}:\n{message.content}\n\n")


### ドラムロール..

## MCPの紹介！

人類からのモデルコンテキストプロトコルを最初に見る - 

Autogenは、Langchainツールと同様に、MCPツールを簡単に使用できます。

<テーブルスタイル= "マージン：0;テキストアライグ：左;幅：100％">
    <tr>
        <td style = "width：150px; height：150px; vertical-align：middle;">
            <img src = "../ assets/stop.png" width = "150" height = "150" style = "display：block;" />
        </td>
        <td>
            <h2 style = "color：＃ff7800;">しかし、待ってください -  Windows PCの人々にとってはそれほど小さな問題</h2>
            <Span style = "color：＃ff7800;">不快なニュースがあります。 Windows PCSでMCPサーバーを実行するのに問題があります。 MacとLinuxは問題ありません。これは、2025年5月4日の既知の問題です。私はO3に深い研究を依頼して、回避策を見つけようとしました。 <a href = "https://chatgpt.com/share/6817bbc3-3d0c-8012-9b51-631842470628">問題を確認し、ワークアラウンドを確認しました。<br/> <br/>
            回避策は少し退屈です。これは、PCでLinuxを実行するためのMicrosoftアプローチである「WSL」を利用することです。より多くのセットアップ手順を実行する必要があります！しかし、それは迅速であり、数人の学生がこれが彼らのために完全に機能することを確認しました、そしてこのラボと週6のMCPラボが機能することを確認しました。さらに、WSLは実際にWindows PCにソフトウェアを構築する素晴らしい方法です。この最終セルをスキップすることもできますが、第6週を開始するときにこれに戻る必要があります。<br/>
            WSLのセットアップ手順は、<a href = "../ setup/setup-wsl.md"> <a href = "../ setup-wsl.mdと呼ばれるファイル</a>にあります。私はこれがあなたを簡単に抑えるだけであることを願っています - あなたはすぐに戻って走るべきです。ああ、出血エッジテクノロジーを扱う喜び！<br/> <br/>
            学生のKaushik R.に感謝します。これはここと6週目です。Kaushikに感謝します。
            </span>
        </td>
    </tr>
</table>

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.mcp import StdioServerParams, mcp_server_tools

# MCP-Server-Fetchからフェッチツールを入手します。
fetch_mcp_server = StdioServerParams(command="uvx", args=["mcp-server-fetch"], read_timeout_seconds=30)
fetcher = await mcp_server_tools(fetch_mcp_server)

# フェッチツールを使用できるエージェントを作成します。
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
agent = AssistantAgent(name="fetcher", model_client=model_client, tools=fetcher, reflect_on_tool_use=True)  # タイプ：無視します

# エージェントにURLのコンテンツを取得し、要約します。
result = await agent.run(task="Review edwarddonner.com and summarize what you learn. Reply in Markdown.")
display(Markdown(result.messages[-1].content))